In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score

In [2]:

def convert_dataturks_to_spacy(filename):

    with open(filename) as train_data:
	
        train = json.load(train_data)

    TRAIN_DATA = []

    for data in train:

	    ents = [tuple(entity) for entity in data['entities']]

	    TRAIN_DATA.append((data['content'],{'entities':ents}))
        
    return TRAIN_DATA

In [3]:
TRAIN_DATA = convert_dataturks_to_spacy('data/data_base/NER_traindata.json')
TRAIN_DATA = TRAIN_DATA[:][:-1]

In [6]:
import thinc_gpu_ops
thinc_gpu_ops.AVAILABLE

#import spacy 
#spacy.prefer_gpu()
#spacy.require_gpu()


True

In [5]:
#import cupy

ImportError: CuPy is not correctly installed.

If you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.
Also, confirm that only one CuPy package is installed:
  $ pip freeze

If you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:
  $ pip install cupy --no-cache-dir -vvvv

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

original error: libcudart.so.10.2: cannot open shared object file: No such file or directory

In [13]:

import time
# check process time 
start_time = time.time()
# Optimal values: n_iter = 10, drop = 0.01

## Hyperparameters
model = None
output_dir=Path("./data/results/models")
n_iter=10

## Load model

#load the model
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

## Disable PIPELINE
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        #print(ent[2])
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.01,  
                sgd=optimizer,
                losses=losses)
        print(losses)
print('.'*50)
print("--- %s seconds ---" % (time.time() - start_time))
print('.'*50)

Created blank 'en' model
100%|██████████| 215/215 [00:04<00:00, 45.97it/s]{'ner': 437.7076096291661}
--- 46.10455632209778 seconds ---



## Save the re-trained model ##

In [14]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to data/results/models
